# ML with TensorFlow Extended (TFX) Pipelines
1. Extracting the new training data from the source using [ExampleGen](https://www.tensorflow.org/tfx/guide/examplegen) component.
2. Generating statistics from the the incoming data using [StatisticsGen](https://www.tensorflow.org/tfx/guide/statsgen) component.
3. Importing a fixed raw schema using [ImporterNode](https://github.com/tensorflow/tfx/blob/master/tfx/components/common_nodes/importer_node.py) component.
4. Validating data based on the schema using [ExampleValidator](https://www.tensorflow.org/tfx/guide/exampleval) component.
5. Transforming the data for ML using the [Transform](https://www.tensorflow.org/tfx/guide/transform) component.
6. Training the model using the [Trainer](https://www.tensorflow.org/tfx/guide/trainer) component.
7. Evaluate the model using the [Evaluator](https://www.tensorflow.org/tfx/guide/evaluator) component.
8. Validate the model using a [Custom TFX](https://www.tensorflow.org/tfx/guide/custom_component) component.
9. Query the [ML Metadata](https://www.tensorflow.org/tfx/guide/mlmd) DB

In [1]:
import os

DATA_DIR='../data'
TRAIN_DATA_DIR = os.path.join(DATA_DIR, 'train')
RAW_SCHEMA_DIR = '../raw_schema'
OUTPUT_DIR='../workspace'

# Training Pipeline

In [2]:
import tfx
import tensorflow as tf
print("Tensorflow Version:", tf.__version__)
print("TFX Version:", tfx.__version__)

Tensorflow Version: 1.15.0
TFX Version: 0.15.0


## 0. Create Interactive Context
This will use an ephemeral SQLite MLMD connection contained in the pipeline_root directory with file name "metadata.sqlite" will be used.

In [3]:
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext

PIPELINE_NAME = 'tfx-census-classification'

context = InteractiveContext(
    pipeline_name=PIPELINE_NAME,
    pipeline_root=OUTPUT_DIR,
    metadata_connection_config=None
)

In [4]:
from pprint import pprint
pprint("Standard Artifact types:")
pprint([a for a in dir(tfx.types.standard_artifacts) if a[0].isupper()])

'Standard Artifact types:'
['ExampleAnomalies',
 'ExampleStatistics',
 'Examples',
 'ExternalArtifact',
 'InferenceResult',
 'Model',
 'ModelBlessing',
 'ModelEvaluation',
 'PushedModel',
 'Schema',
 'TransformGraph']


## 1. Data Ingestion (ExampleGen)
1. Reads the CSV data files (expecting to include headers)
2. Split the data to train and eval sets
3. Write the data to TFRecords


* **Inputs**: ExternalPath
* **Ouptpus**: Examples (TFRecords)
* **Properties**: split ratio

In [5]:
from tfx.utils.dsl_utils import external_input
from tfx.proto import example_gen_pb2

output_config = example_gen_pb2.Output(
    split_config=example_gen_pb2.SplitConfig(splits=[
        example_gen_pb2.SplitConfig.Split(name='train', hash_buckets=4),
        example_gen_pb2.SplitConfig.Split(name='eval', hash_buckets=1)
    ]))


example_gen = tfx.components.CsvExampleGen(
    instance_name='Data_Extraction_Spliting',
    input=external_input(TRAIN_DATA_DIR),
    output_config=output_config
)

context.run(example_gen)

ExecutionResult(
    component_id: CsvExampleGen.Data_Extraction_Spliting
    execution_id: 2
    outputs:
        examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: ../workspace/CsvExampleGen.Data_Extraction_Spliting/examples/1/train/, split: train, id: 2)
            Artifact(type_name: ExamplesPath, uri: ../workspace/CsvExampleGen.Data_Extraction_Spliting/examples/1/eval/, split: eval, id: 3)]
        ))

In [6]:
##Get the URI of the output artifact representing the training examples, which is a directory
# tf.enable_eager_execution()

# import tensorflow_data_validation as tfdv

# train_uri = example_gen.outputs['examples'].get()[0].uri

# tfrecord_filenames = tf.data.Dataset.list_files(train_uri+"*")

# # Create a `TFRecordDataset` to read these files
# dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")

# # Iterate over the first 3 records and decode them using a TFExampleDecoder
# for tfrecord in dataset.take(3):
#     serialized_example = tfrecord.numpy()
#     example = tfdv.TFExampleDecoder().decode(serialized_example)
#     pprint(example)
#     print("")

## 2. Data Validation
1. Generate the **statistics** for the data to validate.
2. Import the **raw_schema** created in the Data Analysis phase.
3. Validat the **statistics** against the schema and generate **anomalies** (if any).

### 2.1. Generating statistics for the data to validate (StatisticsGen)
* **Inputs**: Examples
* **Outputs**: ExampleStatistics
* **Properries**: None

In [7]:
statistics_gen = tfx.components.StatisticsGen(
    instance_name='Statistics_Generation',
    examples=example_gen.outputs['examples'])
context.run(statistics_gen)

ExecutionResult(
    component_id: StatisticsGen.Statistics_Generation
    execution_id: 3
    outputs:
        output: Channel(
            type_name: ExampleStatisticsPath
            artifacts: [Artifact(type_name: ExampleStatisticsPath, uri: ../workspace/StatisticsGen.Statistics_Generation/output/3/train/, split: train, id: 4)
            Artifact(type_name: ExampleStatisticsPath, uri: ../workspace/StatisticsGen.Statistics_Generation/output/3/eval/, split: eval, id: 5)]
        ))

In [8]:
context.show(statistics_gen.outputs['statistics'])

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


### 2.2. Import the fixed raw schema (ImporterNode)
The **ImporterNode** allows you to import an external artifact to a component.
You need to specifiy:
1. Artifact Type
2. Artifcat Location

In [9]:
schema_importer = tfx.components.common_nodes.importer_node.ImporterNode(
    instance_name='Schema_Importer',
    source_uri=RAW_SCHEMA_DIR,
    artifact_type=tfx.types.standard_artifacts.Schema,
    reimport=False
)

context.run(schema_importer)

ExecutionResult(
    component_id: ImporterNode.Schema_Importer
    execution_id: 4
    outputs:
        result: Channel(
            type_name: SchemaPath
            artifacts: [Artifact(type_name: SchemaPath, uri: ../raw_schema, split: , id: 6)]
        ))

In [10]:
context.show(schema_importer.outputs['result'])

,Type,Presence,Valency,Domain
Feature name,,,,
'education',STRING,required,,'education'
'gender',STRING,required,,'gender'
'income_bracket',STRING,required,,'income_bracket'
'marital_status',STRING,required,,'marital_status'
'native_country',STRING,required,,'native_country'
'occupation',STRING,required,,'occupation'
'race',STRING,required,,'race'
'relationship',STRING,required,,'relationship'
'workclass',STRING,required,,'workclass'


,Values
Domain,
'education',"' 10th', ' 11th', ' 12th', ' 1st-4th', ' 5th-6th', ' 7th-8th', ' 9th', ' Assoc-acdm', ' Assoc-voc', ' Bachelors', ' Doctorate', ' HS-grad', ' Masters', ' Preschool', ' Prof-school', ' Some-college'"
'gender',"' Female', ' Male'"
'income_bracket',"' <=50K', ' >50K'"
'marital_status',"' Divorced', ' Married-AF-spouse', ' Married-civ-spouse', ' Married-spouse-absent', ' Never-married', ' Separated', ' Widowed'"
'native_country',"' ?', ' Cambodia', ' Canada', ' China', ' Columbia', ' Cuba', ' Dominican-Republic', ' Ecuador', ' El-Salvador', ' England', ' France', ' Germany', ' Greece', ' Guatemala', ' Haiti', ' Holand-Netherlands', ' Honduras', ' Hong', ' Hungary', ' India', ' Iran', ' Ireland', ' Italy', ' Jamaica', ' Japan', ' Laos', ' Mexico', ' Nicaragua', ' Outlying-US(Guam-USVI-etc)', ' Peru', ' Philippines', ' Poland', ' Portugal', ' Puerto-Rico', ' Scotland', ' South', ' Taiwan', ' Thailand', ' Trinadad&Tobago', ' United-States', ' Vietnam', ' Yugoslavia', 'Egypt'"
'occupation',"' ?', ' Adm-clerical', ' Armed-Forces', ' Craft-repair', ' Exec-managerial', ' Farming-fishing', ' Handlers-cleaners', ' Machine-op-inspct', ' Other-service', ' Priv-house-serv', ' Prof-specialty', ' Protective-serv', ' Sales', ' Tech-support', ' Transport-moving'"
'race',"' Amer-Indian-Eskimo', ' Asian-Pac-Islander', ' Black', ' Other', ' White'"
'relationship',"' Husband', ' Not-in-family', ' Other-relative', ' Own-child', ' Unmarried', ' Wife'"
'workclass',"' ?', ' Federal-gov', ' Local-gov', ' Never-worked', ' Private', ' Self-emp-inc', ' Self-emp-not-inc', ' State-gov', ' Without-pay'"


### 2.3. Validate the input data statistics (ExampleValidator)
* **Inputs**: ExampleStatistics, Schema
* **Outputs**: ExampleAnomalies (if any)
* **Properties**: None

In [11]:
example_validator = tfx.components.ExampleValidator(
    statistics=statistics_gen.outputs['statistics'],
    schema=schema_importer.outputs['result'],
    instance_name="Data_Validation"
)

context.run(example_validator)

ExecutionResult(
    component_id: ExampleValidator.Data_Validation
    execution_id: 5
    outputs:
        output: Channel(
            type_name: ExampleValidationPath
            artifacts: [Artifact(type_name: ExampleValidationPath, uri: ../workspace/ExampleValidator.Data_Validation/output/5/, split: , id: 7)]
        ))

In [12]:
context.show(example_validator.outputs['anomalies'])

## 3. Data Preprocessing

### 3.1. Implement the preprocessing logic

We need to implement the preprocessing logic in a python module: **transform.py**.

This module is expected to have **preprocessing_fn** method, which accept a dictionary of tensors representing the raw features), and return a dictionary of tensors representing the transformed features.

We use the **raw schema** to identify feature types and the required transformation.

The function is implemented using [TensorFlow Transform](https://www.tensorflow.org/tfx/guide/tft).

In [13]:
_transform_module_file = '../src/transform.py'

In [16]:
%%writefile {_transform_module_file}

import tensorflow_transform as tft
import tensorflow_data_validation as tfdv

TARGET_FEATURE_NAME = 'income_bracket'
WEIGHT_FEATURE_NAME = 'fnlwgt'
RAW_SCHEMA_LOCATION = '../raw_schema/schema.pbtxt'

raw_schema = tfdv.load_schema_text(RAW_SCHEMA_LOCATION)

def preprocessing_fn(input_features):

    processed_features = {}

    for feature in raw_schema.feature:
        
        # Pass the target feature as is.
        if feature.name in [TARGET_FEATURE_NAME, WEIGHT_FEATURE_NAME]:
            processed_features[feature.name] = input_features[feature.name]
            continue

        if feature.type == 1:
            # Extract vocabulary and integerize categorical features.
            processed_features[feature.name+"_integerized"] = tft.compute_and_apply_vocabulary(
                input_features[feature.name], vocab_filename=feature.name)
        else:
            # normalize numeric features.
            processed_features[feature.name+"_scaled"] = tft.scale_to_z_score(input_features[feature.name])

        # Bucketize age using quantiles. 
        quantiles = tft.quantiles(input_features["age"], num_buckets=5, epsilon=0.01)
        processed_features["age_bucketized"] = tft.apply_buckets(
            input_features["age"], bucket_boundaries=quantiles)

    return processed_features



Overwriting ../src/transform.py


### 3.2. Tranform train and eval data (Transform)

The component uses the transform output generated from transforming the train data to transform eval data.
That is, while the train data is **analyzed** and **transformed**, the eval data is **only transformed** uaing the output of the analyze phase (TransformGraph) on the train data.

* **Inputs**: train and eval data (Examples), raw schema (Schema), transformation module (file)
* **outputs**: transformed train and eval data (Examples), transform output (TransformGraph)

In [17]:
transform = tfx.components.Transform(
    examples=example_gen.outputs['examples'],
    schema=schema_importer.outputs['result'],
    module_file=_transform_module_file,
    instance_name="Data_Transformation"
)

context.run(transform)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.build_tensor_info or tf.compat.v1.saved_model.build_tensor_info.
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../workspace/Transform.Data_Transformation/transform_output/7/.temp_path/tftransform_tmp/ad3dd005fab048bd89253649348ea6af/saved_model.pb
INFO:tensorflow:Assets added to graph.
INFO:tensorflow:No assets to write.
INFO:tensorflow:SavedModel written to: ../workspace/Transform.Data_Transformation/transform_output/7/.temp_path/tftransform_tmp/7cd5d454ddca44d1ac7bc55907beaad7/saved_model.pb
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Saver not created because there are no variables in the graph to restore
INFO:tensorflow:Assets add

ExecutionResult(
    component_id: Transform.Data_Transformation
    execution_id: 7
    outputs:
        transform_output: Channel(
            type_name: TransformPath
            artifacts: [Artifact(type_name: TransformPath, uri: ../workspace/Transform.Data_Transformation/transform_output/7/, split: , id: 8)]
        )
        transformed_examples: Channel(
            type_name: ExamplesPath
            artifacts: [Artifact(type_name: ExamplesPath, uri: ../workspace/Transform.Data_Transformation/transformed_examples/7/train/, split: train, id: 9)
            Artifact(type_name: ExamplesPath, uri: ../workspace/Transform.Data_Transformation/transformed_examples/7/eval/, split: eval, id: 10)]
        ))

In [18]:
uri = transform.outputs['transform_graph'].get()[0].uri
os.listdir(uri)

['transformed_metadata', 'transform_fn', 'metadata']

In [19]:
# t.enable_eager_execution()

# train_uri = transform.outputs['transformed_examples'].get()[1].uri
# tfrecord_filenames = [os.path.join(train_uri, name)
#                       for name in os.listdir(train_uri)]
# dataset = tf.data.TFRecordDataset(tfrecord_filenames, compression_type="GZIP")
# decoder = tfdv.TFExampleDecoder()
# for tfrecord in dataset.take(3):
#     serialized_example = tfrecord.numpy()
#     example = decoder.decode(serialized_example)
#     pprint(example)
#     print("")

## 4. Train the Model (Trainer)

### 4.1 Implement a train Python module.

Create a Python module containing a **trainer_fn** function, which must return an estimator.

The **trainer_fn** receives the following parameters:
* hparams: currently includes train_steps, eval_steps, and transform_output.
* schema: the raw data schema. This is used to create the serving input function to export the model.

The **trainer_fn** returns a dictionary of the following:
* estimator: The estimator that will be used for training and eval.
* train_spec: Spec for training.
* eval_spec: Spec for eval.
* eval_input_receiver_fn: Input function for eval.

In [20]:
_train_module_file = '../src/train.py'

In [21]:
%%writefile {_train_module_file}

import math

import tensorflow as tf
import tensorflow_transform as tft
import tensorflow_model_analysis as tfma
from tensorflow_transform.tf_metadata import schema_utils


TARGET_FEATURE_NAME = 'income_bracket'
WEIGHT_FEATURE_NAME = 'fnlwgt'
TARGET_FEATURE_LABELS = [' <=50K', ' >50K']

# Input function for train and eval transformed data
def make_input_fn(tfrecords_files, transform_output,
                  batch_size, num_epochs=1, shuffle=False):
    
    def _gzip_reader_fn(filenames):
        return tf.data.TFRecordDataset(
            filenames, compression_type='GZIP')
    
    def input_fn():
        dataset = tf.data.experimental.make_batched_features_dataset(
            file_pattern=tfrecords_files,
            batch_size=batch_size,
            features=transform_output.transformed_feature_spec(),
            label_key=TARGET_FEATURE_NAME,
            reader=_gzip_reader_fn,
            num_epochs=num_epochs,
            shuffle=shuffle
        )
        return dataset
    return input_fn


# Create estimator
def create_estimator(params, transform_output, run_config):
    
    feature_columns = []
    
    # Create feature columns based on the transform schema
    transformed_features = transform_output.transformed_metadata.schema.feature
    for feature in transformed_features:
        
        if feature.name in [TARGET_FEATURE_NAME, WEIGHT_FEATURE_NAME]:
            continue

        if hasattr(feature, 'int_domain') and feature.int_domain.is_categorical:
            vocab_size = feature.int_domain.max + 1
            feature_columns.append(
                tf.feature_column.embedding_column(
                    tf.feature_column.categorical_column_with_identity(
                        feature.name, num_buckets=vocab_size),
                    dimension = int(math.sqrt(vocab_size))))
        else:
            feature_columns.append(
                tf.feature_column.numeric_column(feature.name))
            
    # Create DNNClassifier        
    estimator = tf.estimator.DNNClassifier(
        weight_column=WEIGHT_FEATURE_NAME,
        label_vocabulary=TARGET_FEATURE_LABELS,
        feature_columns=feature_columns,
        hidden_units=params.hidden_units,
        warm_start_from=params.warm_start_from,
        config=run_config
    )
    
    return estimator


# Create serving input function for model serving
def _serving_input_receiver_fn(transform_output, schema):
    
    raw_feature_spec = schema_utils.schema_as_feature_spec(schema).feature_spec
    raw_feature_spec.pop(TARGET_FEATURE_NAME)
    raw_feature_spec.pop(WEIGHT_FEATURE_NAME)
    
#     # Create the interface for the serving function with the raw features
#     serving_input_receiver = tf.estimator.export.build_parsing_serving_input_receiver_fn(
#           raw_feature_spec, default_batch_size=None)()
    
#     # Apply the transform function 
#     transformed_features = transform_output.transform_raw_features(
#         serving_input_receiver.features)
    
#     return tf.estimator.export.ServingInputReceiver(
#         transformed_features, 
#         serving_input_receiver.features
#     )

    # Create the interface for the serving function with the raw features
    raw_features = tf.estimator.export.build_parsing_serving_input_receiver_fn(
        raw_feature_spec)().features

    receiver_tensors = {
        feature: tf.placeholder(shape=[None], dtype=raw_features[feature].dtype) 
        for feature in raw_features
    }

    receiver_tensors_expanded = {
        tensor: tf.reshape(receiver_tensors[tensor], (-1, 1)) 
        for tensor in receiver_tensors
    }

    # Apply the transform function 
    transformed_features = transform_output.transform_raw_features(
        receiver_tensors_expanded)

    return tf.estimator.export.ServingInputReceiver(
        transformed_features, receiver_tensors)


# Create eval input function for model evaluation
def _eval_input_receiver_fn(transform_output, schema):
    
    raw_feature_spec = schema_utils.schema_as_feature_spec(schema).feature_spec
    serving_input_receiver = tf.estimator.export.build_parsing_serving_input_receiver_fn(
        raw_feature_spec, default_batch_size=None)()
    
    features = serving_input_receiver.features.copy()
    transformed_features = transform_output.transform_raw_features(features)

    features.update(transformed_features)
    
    return tfma.export.EvalInputReceiver(
        features=features,
        receiver_tensors=serving_input_receiver.receiver_tensors,
        labels=transformed_features[TARGET_FEATURE_NAME]
    )



# TFX will call this function
def trainer_fn(hparams, schema):

    hidden_units = [128, 64]
    batch_size = 40
    train_steps = hparams.train_steps
    eval_steps =  hparams.eval_steps
    model_dir = hparams.serving_model_dir
    
    hparams.hidden_units = hidden_units
    
    # Load TFT Transform Output
    transform_output = tft.TFTransformOutput(hparams.transform_output)
    
    # Create Train Sepc
    train_spec = tf.estimator.TrainSpec(
        input_fn = make_input_fn(
            hparams.train_files,
            transform_output,
            batch_size=batch_size,
            num_epochs=None,
            shuffle=True
      ),
      max_steps=train_steps
    )
    
    # Create Exportor with the serving receiver function
    serving_receiver_fn = lambda: _serving_input_receiver_fn(transform_output, schema)
    exporter = tf.estimator.FinalExporter('census', serving_receiver_fn)
    
    # Create Eval Sepc
    eval_spec = tf.estimator.EvalSpec(
        input_fn = make_input_fn(
            hparams.eval_files,
            transform_output,
            batch_size=batch_size
        ),
        exporters=[exporter],
        start_delay_secs=0,
        throttle_secs=0,
        steps=eval_steps,
        name='census-eval'
    )
    
    # Create Run Config
    run_config = tf.estimator.RunConfig(
        tf_random_seed=19831006,
        save_checkpoints_steps=200, 
        keep_checkpoint_max=3, 
        model_dir=model_dir,
        log_step_count_steps=10
    )
    
    # Create estimator
    estimator = create_estimator(hparams, transform_output, run_config)

    # Create an input receiver for TFMA processing
    eval_receiver_fn = lambda: _eval_input_receiver_fn(
        transform_output, schema)
    
    return {
        'estimator': estimator,
        'train_spec': train_spec,
        'eval_spec': eval_spec,
        'eval_input_receiver_fn': eval_receiver_fn
    }

Overwriting ../src/train.py


### 4.2 Train the model using the Trainer component
* **Inputs**: train module file with the **trainer_fn**, raw schema (Schema), and transform output (TransformGraph)
* **Outputs**: saved_model (Model)
* **Properties**: train and eval args


In [22]:
trainer = tfx.components.Trainer(
    module_file=_train_module_file,
    transformed_examples=transform.outputs['transformed_examples'],
    schema=schema_importer.outputs['result'],
    transform_graph=transform.outputs['transform_graph'],
    train_args=tfx.proto.trainer_pb2.TrainArgs(num_steps=1000),
    eval_args=tfx.proto.trainer_pb2.EvalArgs(num_steps=None),
    instance_name='Census_Classifier_Trainer'
)

context.run(trainer)

INFO:tensorflow:Using config: {'_model_dir': '../workspace/Trainer.Census_Classifier_Trainer/output/8/serving_model_dir', '_tf_random_seed': 19831006, '_save_summary_steps': 100, '_save_checkpoints_steps': 200, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 10, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_session_creation_timeout_secs': 7200, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fb58147e7f0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}
INFO:tensorflow:Not using Distribu

INFO:tensorflow:Calling model_fn.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Starting evaluation at 2019-11-14T20:15:33Z
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ../workspace/Trainer.Census_Classifier_Trainer/output/8/serving_model_dir/model.ckpt-200
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Finished evaluation at 2019-11-14-20:15:35
INFO:tensorflow:Saving dict for global step 200: accuracy = 0.8550376, accuracy_baseline = 0.76336056, auc = 0.91003406, auc_precision_recall = 0.77584726, average_loss = 0.31128925, global_step = 200, label/mean = 0.23663943, loss = 2359360.2, precision = 0.75586057, prediction/mean = 0.22752647, recall = 0.5722435
INFO:tensorflow:Saving 'checkpoint_path' summary for global step 200: ../workspace/Trainer.Census_Classifier_Trainer/output/8/serving_model_dir/model.ckpt-200
IN

INFO:tensorflow:Saving 'checkpoint_path' summary for global step 600: ../workspace/Trainer.Census_Classifier_Trainer/output/8/serving_model_dir/model.ckpt-600
INFO:tensorflow:global_step/sec: 3.54415
INFO:tensorflow:loss = 2509643.0, step = 601 (2.822 sec)
INFO:tensorflow:global_step/sec: 641.282
INFO:tensorflow:loss = 2236766.2, step = 611 (0.015 sec)
INFO:tensorflow:global_step/sec: 628.332
INFO:tensorflow:loss = 3453494.0, step = 621 (0.016 sec)
INFO:tensorflow:global_step/sec: 590.564
INFO:tensorflow:loss = 1867877.4, step = 631 (0.017 sec)
INFO:tensorflow:global_step/sec: 615.913
INFO:tensorflow:loss = 2216915.0, step = 641 (0.016 sec)
INFO:tensorflow:global_step/sec: 627.467
INFO:tensorflow:loss = 1705023.6, step = 651 (0.016 sec)
INFO:tensorflow:global_step/sec: 622.975
INFO:tensorflow:loss = 2830259.8, step = 661 (0.016 sec)
INFO:tensorflow:global_step/sec: 609.416
INFO:tensorflow:loss = 2204543.0, step = 671 (0.016 sec)
INFO:tensorflow:global_step/sec: 619.781
INFO:tensorflow:

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Signatures INCLUDED in export for Classify: None
INFO:tensorflow:Signatures INCLUDED in export for Regress: None
INFO:tensorflow:Signatures INCLUDED in export for Predict: ['predict']
INFO:tensorflow:Signatures INCLUDED in export for Train: None
INFO:tensorflow:Signatures INCLUDED in export for Eval: None
INFO:tensorflow:Signatures EXCLUDED from export because they cannot be be served via TensorFlow Serving APIs:
INFO:tensorflow:'serving_default' : Classification input must be a single string Tensor; got {'age': <tf.Tensor 'Placeholder:0' shape=(?,) dtype=int64>, 'capital_gain': <tf.Tensor 'Placeholder_1:0' shape=(?,) dtype=int64>, 'capital_loss': <tf.Tensor 'Placeholder_2:0' shape=(?,) dtype=int64>, 'education': <tf.Tensor 'Placeholder_3:0' shape=(?,) dtype=string>, 'education_num': <tf.Tensor 'Placeholder_4:0' shape=(?,) dtype=int64>, 'gender': <tf.Tensor 'Placeholder_5:0' shape=(?,) dtype=string

ExecutionResult(
    component_id: Trainer.Census_Classifier_Trainer
    execution_id: 8
    outputs:
        output: Channel(
            type_name: ModelExportPath
            artifacts: [Artifact(type_name: ModelExportPath, uri: ../workspace/Trainer.Census_Classifier_Trainer/output/8/, split: , id: 11)]
        ))

In [23]:
train_uri = trainer.outputs['model'].get()[0].uri
serving_model_path = os.path.join(train_uri, 'serving_model_dir', 'export', 'census')
latest_serving_model_path = os.path.join(serving_model_path, max(os.listdir(serving_model_path)))
print(latest_serving_model_path)

../workspace/Trainer.Census_Classifier_Trainer/output/8/serving_model_dir/export/census/1573762548


In [24]:
!saved_model_cli show --dir={latest_serving_model_path} --all


MetaGraphDef with tag-set: 'serve' contains the following SignatureDefs:

signature_def['predict']:
  The given SavedModel SignatureDef contains the following input(s):
    inputs['age'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder:0
    inputs['capital_gain'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder_1:0
    inputs['capital_loss'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder_2:0
    inputs['education'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: Placeholder_3:0
    inputs['education_num'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder_4:0
    inputs['gender'] tensor_info:
        dtype: DT_STRING
        shape: (-1)
        name: Placeholder_5:0
    inputs['hours_per_week'] tensor_info:
        dtype: DT_INT64
        shape: (-1)
        name: Placeholder_6:0
    inputs['marital_status'] tensor_info:
     

### 4.3. Use exported model for prediction

In [25]:
predictor_fn = tf.contrib.predictor.from_saved_model(
    export_dir = latest_serving_model_path,
    signature_def_key="predict"
)
print("")

output = predictor_fn(
    {
        'age': [34.0],
        'workclass': ['Private'],
        'education': ['Doctorate'],
        'education_num': [10.0],
        'marital_status': ['Married-civ-spouse'],
        'occupation': ['Prof-specialty'],
        'relationship': ['Husband'],
        'race': ['White'],
        'gender': ['Male'],
        'capital_gain': [0.0], 
        'capital_loss': [0.0], 
        'hours_per_week': [40.0],
        'native_country':['Egyptian']
    }
)

print(output)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.loader.load or tf.compat.v1.saved_model.load. There will be a new function for importing SavedModels in Tensorflow 2.0.
INFO:tensorflow:Restoring parameters from ../workspace/Trainer.Census_Classifier_Trainer/output/8/serving_model_dir/export/census/1573762548/variables/variables

{'class_ids': array([[0]]), 'classes': array([[b' <=50K']], dtype=object), 'all_class_ids': array([[0, 1]], dtype=int32), 'all_classes': array([[b' <=50K', b' >50K']], dtype=object), 'logistic': array([[0.1901608]], dtype=flo

## 5. Evaluate the trained model (Evaluator)
* **Inputs**: eval data (Examples), trained model (Model)
* **Outputs** eval metric (ModelEvaluation)
* **Properties**: Slicing Specs

In [26]:
feature_slicing_spec=tfx.proto.evaluator_pb2.FeatureSlicingSpec(
    specs=[
        tfx.proto.evaluator_pb2.SingleSlicingSpec(),
        tfx.proto.evaluator_pb2.SingleSlicingSpec(column_for_slicing=['occupation'])
    ]
)


model_analyzer = tfx.components.Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    feature_slicing_spec=feature_slicing_spec,
    instance_name="Occupation_based_Evaluator"
)

context.run(model_analyzer)


INFO:tensorflow:Restoring parameters from ../workspace/Trainer.Census_Classifier_Trainer/output/8/eval_model_dir/1573762549/variables/variables
Instructions for updating:
This function will only be available through the v1 compatibility library as tf.compat.v1.saved_model.utils.get_tensor_from_tensor_info or tf.compat.v1.saved_model.get_tensor_from_tensor_info.


ExecutionResult(
    component_id: Evaluator.Occupation_based_Evaluator
    execution_id: 9
    outputs:
        output: Channel(
            type_name: ModelEvalPath
            artifacts: [Artifact(type_name: ModelEvalPath, uri: ../workspace/Evaluator.Occupation_based_Evaluator/output/9/, split: , id: 12)]
        ))

In [29]:
import tensorflow_model_analysis as tfma

results_uri = model_analyzer.outputs['output'].get()[0].uri
eval_result = tfma.load_eval_result(results_uri)
eval_result.slicing_metrics

[((),
  {'': {'': {'auc': {'doubleValue': 0.9149421453475952},
     'auc_precision_recall': {'doubleValue': 0.7889833450317383},
     'accuracy': {'doubleValue': 0.858815610408783},
     'average_loss': {'doubleValue': 0.30137526988983154},
     'recall': {'doubleValue': 0.6057470440864563},
     'label/mean': {'doubleValue': 0.23663939535617828},
     'post_export_metrics/example_count': {'doubleValue': 9545.0},
     'precision': {'doubleValue': 0.7495791912078857},
     'prediction/mean': {'doubleValue': 0.23269863426685333},
     'accuracy_baseline': {'doubleValue': 0.7633606195449829}}}}),
 ((('occupation', ' Prof-specialty'),),
  {'': {'': {'post_export_metrics/example_count': {'doubleValue': 1262.0},
     'precision': {'doubleValue': 0.8198577761650085},
     'accuracy_baseline': {'doubleValue': 0.5359025001525879},
     'prediction/mean': {'doubleValue': 0.43871957063674927},
     'auc': {'doubleValue': 0.8974887132644653},
     'auc_precision_recall': {'doubleValue': 0.88712167

In [30]:
tfma.view.render_slicing_metrics(
    result=eval_result, 
    slicing_column='occupation'
)

SlicingMetricsViewer(config={'weightedExamplesColumn': 'post_export_metrics/example_count'}, data=[{'slice': '…

## 6. Validate the Trained Model

In [31]:
model_validator = tfx.components.ModelValidator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    instance_name="Base_Model_Validator"
)
context.run(model_validator)

INFO:tensorflow:Restoring parameters from ../workspace/Trainer.Census_Classifier_Trainer/output/8/eval_model_dir/1573762549/variables/variables


ExecutionResult(
    component_id: ModelValidator.Base_Model_Validator
    execution_id: 10
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: ../workspace/ModelValidator.Base_Model_Validator/blessing/10/, split: , id: 13)]
        ))

We will create a Custom TFX Component that validates the trained model based on its produced evaluation metric.

The custom validator will **bless** the model if:
1. Overal accuracy is greater than 85%.
2. Accuracy per **Occupation** slice is at most 10% less than the overall accuracy.

* **Inputs**: Evaluation Metric (ModelEvaluation), trained model (Model)
* **Outputs**: blessing (ModelBlessing)
* **Properties**: accuracy_threshold, slice_accuracy_tolerance

### 6.1. Create ComponentSepc

In [40]:
from tfx.types import standard_artifacts
from tfx.types.component_spec import ChannelParameter
from tfx.types.component_spec import ExecutionParameter

class AccuracyValidatorSpec(tfx.types.ComponentSpec):
    
    INPUTS = {
        'eval_results': ChannelParameter(type=standard_artifacts.ModelEvaluation),
        'model': ChannelParameter(type=standard_artifacts.Model),
    }
    
    OUTPUTS = {
      'blessing': ChannelParameter(type=standard_artifacts.ModelBlessing),
    }
    
    PARAMETERS = {
        'accuracy_threshold': ExecutionParameter(type=float),
        'slice_accuracy_tolerance': ExecutionParameter(type=float),
    }


## 6.2. Create Custom Exectutor

In [65]:
from tfx.components.base import base_executor
from tfx.types import artifact_utils
from tfx.utils import io_utils

class AccuracyValidatorExecutor(base_executor.BaseExecutor):
    
    def Do(self, input_dict, output_dict, exec_properties):
        
        valid = True
        
        self._log_startup(input_dict, output_dict, exec_properties)
        
        accuracy_threshold = exec_properties['accuracy_threshold']
        slice_accuracy_tolerance = exec_properties['slice_accuracy_tolerance']
        min_slice_accuracy = accuracy_threshold - slice_accuracy_tolerance
        print("Accuracy Threshold:", accuracy_threshold)
        print("Slice Accuracy Tolerance:", slice_accuracy_tolerance)
        print("Min Accuracy per Slice:", min_slice_accuracy)
        
        results_uri = input_dict['eval_results'][0].uri
        eval_results = tfma.load_eval_result(results_uri)
        
        overall_acc = eval_result.slicing_metrics[0][1]['']['']['accuracy']['doubleValue']
        print("Overall accuracy:", overall_acc)
        
        if overall_acc >= accuracy_threshold:
            for slicing_metric in eval_result.slicing_metrics:
                slice_acc = slicing_metric[1]['']['']['accuracy']['doubleValue']
                if slice_acc < min_slice_accuracy:
                    print("Slice accuracy value < min accuracy:", slice_acc )
                    valid = False
                    break
        else:
            valid = False
        
        print("Valid:", valid)
        
        blessing = artifact_utils.get_single_instance(output_dict['blessing'])
        
        # Current model.
        current_model = artifact_utils.get_single_instance(input_dict['model'])
        blessing.set_string_custom_property('current_model', current_model.uri)
        blessing.set_int_custom_property('current_model_id', current_model.id)

        if valid:
            io_utils.write_string_file(os.path.join(blessing.uri, 'BLESSED'), '')
            blessing.set_int_custom_property('blessed', 1)
        else:
            io_utils.write_string_file(os.path.join(blessing.uri, 'NOT_BLESSED'), '')
            blessing.set_int_custom_property('blessed', 0)


### 6.3. Create AccuracyModelValidator

In [66]:
from typing import Optional
from tfx import types
from tfx.components.base import base_component
from tfx.components.base import executor_spec

class AccuracyModelValidator(base_component.BaseComponent):

    SPEC_CLASS = AccuracyValidatorSpec
    EXECUTOR_SPEC = executor_spec.ExecutorClassSpec(AccuracyValidatorExecutor)
    
    def __init__(self,
                 eval_results: types.channel,
                 model: types.channel,
                 accuracy_threshold: float,
                 slice_accuracy_tolerance: float,
                 blessing: Optional[types.Channel] = None,
                 instance_name=None):
        
        blessing = blessing or types.Channel(
            type=standard_artifacts.ModelBlessing,
            artifacts=[standard_artifacts.ModelBlessing()])
        
        spec = AccuracyValidatorSpec(
            eval_results=eval_results, model=model, blessing=blessing, 
            accuracy_threshold=accuracy_threshold, slice_accuracy_tolerance=slice_accuracy_tolerance
        )
        
        super().__init__(spec=spec, instance_name=instance_name)
    

In [111]:
accuracy_model_validator = AccuracyModelValidator(
    eval_results=model_analyzer.outputs['output'],
    model=trainer.outputs['model'],
    accuracy_threshold=0.85,
    slice_accuracy_tolerance=0.1,
    instance_name="Accuracy_Model_Validator"
)

context.run(accuracy_model_validator)

ExecutionResult(
    component_id: AccuracyModelValidator.Accuracy_Model_Validator
    execution_id: 41
    outputs:
        blessing: Channel(
            type_name: ModelBlessingPath
            artifacts: [Artifact(type_name: ModelBlessingPath, uri: ../workspace/AccuracyModelValidator.Accuracy_Model_Validator/blessing/20/, split: , id: 18)]
        ))

In [112]:
blessing_uri = accuracy_model_validator.outputs.blessing.get()[0].uri
!ls -l {blessing_uri}

total 0
-rw-r--r--  1 khalidsalama  primarygroup  0 Nov 14 20:23 BLESSED


## 7. Querying Metadata database

In [115]:
import sqlite3
connection = sqlite3.connect(os.path.join(OUTPUT_DIR, 'metadata.sqlite'))
cursor = connection.cursor()

### List tables

In [118]:
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
pprint(cursor.fetchall())

[('Type',),
 ('sqlite_sequence',),
 ('TypeProperty',),
 ('Artifact',),
 ('ArtifactProperty',),
 ('Execution',),
 ('ExecutionProperty',),
 ('Event',),
 ('EventPath',),
 ('MLMDEnv',),
 ('Context',),
 ('ContextProperty',),
 ('Association',),
 ('Attribution',)]


### Query Artifact table

In [122]:
cursor.execute("SELECT * FROM Artifact;")
pprint(cursor.fetchall())

[(1, 1, '../data/train'),
 (2,
  4,
  '../workspace/CsvExampleGen.Data_Extraction_Spliting/examples/1/train/'),
 (3, 4, '../workspace/CsvExampleGen.Data_Extraction_Spliting/examples/1/eval/'),
 (4, 6, '../workspace/StatisticsGen.Statistics_Generation/output/3/train/'),
 (5, 6, '../workspace/StatisticsGen.Statistics_Generation/output/3/eval/'),
 (6, 7, '../raw_schema'),
 (7, 10, '../workspace/ExampleValidator.Data_Validation/output/5/'),
 (8, 12, '../workspace/Transform.Data_Transformation/transform_output/7/'),
 (9,
  4,
  '../workspace/Transform.Data_Transformation/transformed_examples/7/train/'),
 (10,
  4,
  '../workspace/Transform.Data_Transformation/transformed_examples/7/eval/'),
 (11, 14, '../workspace/Trainer.Census_Classifier_Trainer/output/8/'),
 (12, 16, '../workspace/Evaluator.Occupation_based_Evaluator/output/9/'),
 (13, 18, '../workspace/ModelValidator.Base_Model_Validator/blessing/10/'),
 (14,
  18,
  '../workspace/AccuracyModelValidator.Accuracy_Model_Validator/blessing